In [72]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
# from catboost import CatBoostClassifier  # Import CatBoost
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import warnings
from sklearn.ensemble import BaggingClassifier
import lightgbm as lgb


warnings.filterwarnings('ignore')


In [73]:
# Load and preprocess the training data
df = pd.read_csv('train.csv')

# Apply one-hot encoding to categorical columns
df = pd.get_dummies(df)

df_test = pd.read_csv('test.csv')

# Apply the same one-hot encoding to test data
df_test = pd.get_dummies(df_test)


In [74]:
# Drop columns as needed
text_to_find = 'noninvasive'
columns_to_drop = [col for col in df.columns if text_to_find in col]
df.drop(columns=columns_to_drop, inplace=True)
df_test.drop(columns=columns_to_drop, inplace=True)

In [75]:
ob_col= df.select_dtypes(include=['object']).columns
not_ob_col= df.select_dtypes(exclude=['object']).columns
not_ob_col_age= df.select_dtypes(exclude=['object']).columns

not_ob_col.delete([-1])
ob_col

Index([], dtype='object')

In [76]:
scaler = RobustScaler( )
for i in not_ob_col:
    df[i]= scaler.fit_transform(df[i].values.reshape(-1, 1))

not_ob_col_test= df_test.select_dtypes(exclude=['object']).columns
not_ob_col_test.delete(0)
for i in not_ob_col_test:
    df_test[i]= scaler.fit_transform(df_test[i].values.reshape(-1, 1))

In [77]:
for i in not_ob_col_age:
    mean_val=df[i].median()
    df[i].fillna(mean_val,inplace=True)

not_ob_col_age_test= df_test.select_dtypes(exclude=['object']).columns
for i in not_ob_col_age_test:
    mean_val=df_test[i].median()
    df_test[i].fillna(mean_val,inplace=True)

In [78]:
imr = KNNImputer(n_neighbors=10001, weights='distance')
imr = imr.fit(df.values)
df[:] = imr.transform(df.values)
imr_test = KNNImputer(n_neighbors=10000, weights='distance')
imr_test = imr_test.fit(df_test.values)
df_test[:] = imr_test.transform(df_test.values)

In [79]:
X = df.drop(columns='hospital_death')
y = df['hospital_death']
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Check if 'hospital_death' is in numeric_columns before dropping it
if 'hospital_death' in numeric_columns:
    numeric_columns = numeric_columns.drop(['RecordID', 'hospital_id', 'icu_id', 'hospital_death'])
else:
    # Handle the case where 'hospital_death' is not in numeric_columns
    print("Warning: 'hospital_death' not found in numeric_columns")

In [80]:
rbs = RobustScaler()
X[numeric_columns] = rbs.fit_transform(X[numeric_columns])
df_test[numeric_columns] = rbs.transform(df_test[numeric_columns])

In [81]:
# Specify the LightGBM parameters
lgbm_params = {
    'learning_rate': 0.1,
    'max_depth': 3,
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'reg_alpha': 0.9,
    'reg_lambda': 0.8,
    'n_jobs': -1,
    'num_leaves': 10
}

# Create a LightGBMClassifier as the base estimator
base_lgbm_model = lgb.LGBMClassifier(n_estimators=500, random_state=42, **lgbm_params)

# Create a BaggingClassifier with LightGBM as the base estimator
bagging_lgbm_model = BaggingClassifier(base_estimator=base_lgbm_model, n_estimators=200, random_state=42)


In [82]:

# Fit the bagging model on the training data
bagging_lgbm_model.fit(X, y)

[LightGBM] [Info] Number of positive: 4338, number of negative: 45662
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004827 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3985
[LightGBM] [Info] Number of data points in the train set: 50000, number of used features: 87
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.089600 -> initscore=-2.318529
[LightGBM] [Info] Start training from score -2.318529
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

KeyboardInterrupt: 

In [ ]:
# Make predictions on the test data using the bagging model
bagging_predictions = bagging_lgbm_model.predict_proba(df_test)[:, 1]


In [ ]:
# Create a DataFrame with bagging model predictions and record IDs
df_predictions = pd.DataFrame(bagging_predictions, columns=['hospital_death'])
custom_starting_index = 50001
df_predictions.insert(0, 'RecordID', range(custom_starting_index, custom_starting_index + len(df_predictions)))

# Specify the file path for saving the CSV file
csv_file_path = 'prediction_bagging_lgbm.csv'

# Save the bagging model predictions to a CSV file
df_predictions.to_csv(csv_file_path, index=False)